# Benchmark for CPU vs GPU (For Machine Learning):

I decided to see the performance difference between training on a CPU vs GPU.

I used code from https://www.analyticsvidhya.com/blog/2021/11/benchmarking-cpu-and-gpu-performance-with-tensorflow/ , and will put in any notes that I make from this small experiment.

Importing the necessary libraries

In [1]:
#Importing libraries into the session:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

Testing if TensorFlow is running on the GPU, by listing all the physical devices that can be used:

In [2]:
tf.config.experimental.list_physical_devices()

2022-01-18 14:35:16.715512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 14:35:17.499554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 14:35:17.500155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Testing if I am using CUDA on the GPU (using a GTX 950 in this test)

In [3]:
tf.test.is_built_with_cuda()

True

Loading in a large dataset (This is a keras dataset).

This uses the ***famous*** cifar10 dataset, which includes 32*32 colour images, of which is seperated into 50,000 train and 10,000 test images.

In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

Checking the image shape. Also checking the data to see if it is as expected.

In [5]:
# checking images shape
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

Making sure the database is working by querying the first entry

In [6]:
# display single image shape
X_train[0].shape

(32, 32, 3)

Making sure that the labels are as expected

In [7]:
# checking labels
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [8]:
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [9]:
# one hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

Using a function to build a neural netowrk with architecture. Compiling is also included.

In [10]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


Testing the CPU in a 10 epoch run (This was run on a i7-2600)

Looking at telemetry, the CPU was pinned the entire time it was training (100%), and the system was hardly usable.

In [11]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

2022-01-18 13:49:53.958496: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-18 13:49:53.960067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 13:49:53.960474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 13:49:53.960807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

Epoch 1/10
1563/1563 [==============================] - 158s 101ms/step - loss: 1.8100 - accuracy: 0.3567
Epoch 2/10
1563/1563 [==============================] - 154s 99ms/step - loss: 1.6244 - accuracy: 0.4256
Epoch 3/10
1563/1563 [==============================] - 137s 87ms/step - loss: 1.5440 - accuracy: 0.4573
Epoch 4/10
1563/1563 [==============================] - 153s 98ms/step - loss: 1.4844 - accuracy: 0.4771
Epoch 5/10
1563/1563 [==============================] - 163s 104ms/step - loss: 1.4342 - accuracy: 0.4972
Epoch 6/10
1563/1563 [==============================] - 133s 85ms/step - loss: 1.3906 - accuracy: 0.5101
Epoch 7/10
1563/1563 [==============================] - 127s 81ms/step - loss: 1.3512 - accuracy: 0.5260
Epoch 8/10
1563/1563 [==============================] - 126s 81ms/step - loss: 1.3182 - accuracy: 0.5357
Epoch 9/10
1563/1563 [==============================] - 144s 92ms/step - loss: 1.2838 - accuracy: 0.5495
Epoch 10/10
1563/1563 [==============================

Testing the GPU in a 10 epoch run (This was run on a GTX 950)

In [11]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

2022-01-18 14:35:24.337671: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-18 14:35:24.375531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 14:35:24.376219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-18 14:35:24.376589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

Epoch 1/10
1563/1563 [==============================] - 16s 6ms/step - loss: 1.8149 - accuracy: 0.3565
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.6262 - accuracy: 0.4252
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.5435 - accuracy: 0.4555
Epoch 4/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.4852 - accuracy: 0.4757
Epoch 5/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.4343 - accuracy: 0.4936
Epoch 6/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3880 - accuracy: 0.5112
Epoch 7/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3524 - accuracy: 0.5264
Epoch 8/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3150 - accuracy: 0.5368
Epoch 9/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.2856 - accuracy: 0.5490
Epoch 10/10
1563/1563 [==============================] - 10s 6ms/step - l